In [224]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import datetime as dt
import uuid
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
from tqdm import tqdm

In [225]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

In [226]:
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title

In [227]:
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [228]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
dates_objects[prev_day-1].click()

In [229]:
# # gettting all available tables
# table_data = driver.find_elements(By.XPATH, "//table")

In [230]:
# # event
# table_data[0].get_attribute('outerHTML')

In [231]:
# table_data = driver.find_elements(By.XPATH, "//table")

In [232]:
# # event data
# table_data[1].get_attribute('outerHTML')

In [233]:
# # coefficients
# table_data[2].get_attribute('outerHTML')

In [234]:
# # all three tags above are recursively repeated
# table_data[3].get_attribute('outerHTML')

In [235]:
# # another table
# table_data[-2].get_attribute("outerHTML")

In [236]:
# # calendar
# table_data[-1].get_attribute("outerHTML")

In [237]:
def get_event_data(event_data):
    # Event data
    soup = bs(event_data, 'html.parser')
    tds = soup.find_all('td')
    # date, players
    splitted_str = tds[0].text.split('  ')
    event_date = splitted_str[0].strip()
    event_players = splitted_str[1][:-1].strip()
    # event
    event = tds[1].text.split(' Теннис :  ')[1].strip()
    # result
    try:
        result = tds[2].text.split('\t')[2]
        splitted_result = result.split(', ')
        score = splitted_result[0].strip()
        winner = splitted_result[1][:-1].strip()
    except IndexError:
        score, winner = None, None
    return event_date, event_players, event, score, winner


def get_coefficients_data(table_data):
    # coefficients data 
    soup = bs(table_data, 'html.parser')
    columns = [x.text.strip() for x in soup.find_all('th')]
    table_data = soup.find_all('td')
    rows = []
    for i in range(0, len(table_data), len(columns)):
        row = table_data[i-len(columns):i]
        rows.append(row)
    rows = np.array(rows[1:], dtype=object)
    data_function = np.vectorize(lambda x: x.text.strip())
    rows = data_function(rows)
    df_event = pd.DataFrame(columns=columns, data=rows)
    nan_value = float("NaN")
    df_event.replace("", nan_value, inplace=True)
    df_event.dropna(how='all', axis=1, inplace=True)
    return df_event

In [238]:
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from time import sleep

# retry_numbers = 20
# df_current_bc_events = pd.DataFrame()
# dict_current_bc_odds = {}
# npages = driver.find_elements(By.CLASS_NAME, 'pages')
# for page in npages:
#     skip = False
#     for _ in range(retry_numbers):
#         try:
#             driver.execute_script("arguments[0].click();", page)
#         except StaleElementReferenceException as e:
#             skip = True
#         else:
#             break
#     if skip:
#         continue
#     # current page parsing
#     page_events = []
#     dict_page_odds = {}
#     table_data = driver.find_elements(By.XPATH, "//table")
#     for table in tqdm(table_data):
#         id_ = str(uuid.uuid4())[:8]
#         skip = False
#         for _ in range(retry_numbers):
#             try:
#                 table_html = table.get_attribute('outerHTML')
#             except WebDriverException as e:
#                 skip = True
#             else:
#                 break
#         if skip:
#             continue
#         if 'class="event"' in table_html:
#             page_events.append(tuple([id_])+get_event_data(table_html))
#         if 'Дата-время скана линии' in table_html :
#             dict_page_odds[id_] = get_coefficients_data(table_html)
#     df_page_events = pd.DataFrame(
#         columns=['id', 'date', 'players', 'event', 'score', 'winner'],
#         data=page_events
#     )
#     df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
#     dict_current_bc_odds.update(dict_page_odds)

In [239]:
# from time import sleep

# df_current_bc_events = pd.DataFrame()
# dict_current_bc_odds = {}
# npages = driver.find_elements(By.CLASS_NAME, 'pages')
# # driver.execute_script("arguments[0].click();", page)

# # current page parsing
# page_events = []
# dict_page_odds = {}
# table_data = driver.find_elements(By.XPATH, "//table")
# for table in tqdm(table_data):
#     id_ = str(uuid.uuid4())[:8]
#     table_html = table.get_attribute('outerHTML')
#     if 'class="event"' in table_html:
#         page_events.append(tuple([id_])+get_event_data(table_html))
#     if 'Дата-время скана линии' in table_html :
#         dict_page_odds[id_] = get_coefficients_data(table_html)
# df_page_events = pd.DataFrame(
#     columns=['id', 'date', 'players', 'event', 'score', 'winner'],
#     data=page_events
# )
# df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
# dict_current_bc_odds.update(dict_page_odds)


# driver.execute_script("arguments[0].click();", npages[1])
# page_events = []
# dict_page_odds = {}
# table_data = driver.find_elements(By.XPATH, "//table")
# for i in tqdm(range(len(table_data))):
#     id_ = str(uuid.uuid4())[:8]
#     retry = True
#     while retry:
#         try:
#             table_html = table_data[i].get_attribute('outerHTML')
#         except:
#             table_data = driver.find_elements(By.XPATH, "//table")
#         else:
#             retry = False
#     if 'class="event"' in table_html:
#         page_events.append(tuple([id_])+get_event_data(table_html))
#     if 'Дата-время скана линии' in table_html :
#         dict_page_odds[id_] = get_coefficients_data(table_html)
# df_page_events = pd.DataFrame(
#     columns=['id', 'date', 'players', 'event', 'score', 'winner'],
#     data=page_events
# )
# df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
# dict_current_bc_odds.update(dict_page_odds)

In [240]:
# df_current_bc_events = pd.DataFrame()
# dict_current_bc_odds = {}
# npages = driver.find_elements(By.CLASS_NAME, 'pages')
# # pages iterating
# for i in range(len(npages)):
#     try:
#         driver.execute_script("arguments[0].click();", npages[i])
#     except StaleElementReferenceException:
#         npages = driver.find_elements(By.CLASS_NAME, 'pages')
#         driver.execute_script("arguments[0].click();", npages[i])
#     print([npages[i].text for i in range(len(npages))])
#     # iterating over all tables on page
#     page_events = []
#     dict_page_odds = {}
#     table_data = driver.find_elements(By.XPATH, "//table")
#     temp = table_data.copy()
#     print(len(table_data))
#     for j in tqdm(range(len(table_data))):
#         id_ = str(uuid.uuid4())[:8]
#         try:
#             table_html = table_data[j].get_attribute('outerHTML')
#         except (StaleElementReferenceException, WebDriverException):
#             table_data = driver.find_elements(By.XPATH, "//table")
#             table_html = table_data[j].get_attribute('outerHTML')
#         except IndexError:
#             break
#         if 'class="event"' in table_html:
#             page_events.append(tuple([id_])+get_event_data(table_html)+tuple([str(i+1)]))
#         if 'Дата-время скана линии' in table_html:
#             dict_page_odds[id_] = get_coefficients_data(table_html)
#     df_page_events = pd.DataFrame(
#         columns=['id', 'date', 'players', 'event', 'score', 'winner', 'bc_page'],
#         data=page_events
#     )
#     df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
#     dict_current_bc_odds.update(dict_page_odds)

In [241]:
# df_current_bc_events = pd.DataFrame()
# dict_current_bc_odds = {}
# npages = driver.find_elements(By.CLASS_NAME, 'pages')
# # pages iterating

# # 1
# i = 0
# try:
#     driver.execute_script("arguments[0].click();", npages[i])
# except StaleElementReferenceException:
#     npages = driver.find_elements(By.CLASS_NAME, 'pages')
#     driver.execute_script("arguments[0].click();", npages[i])
# # iterating over all tables on page
# page_events = []
# dict_page_odds = {}
# table_data = driver.find_elements(By.XPATH, "//table")
# temp = table_data.copy()
# print(len(table_data))
# for j in tqdm(range(len(table_data))):
#     id_ = str(uuid.uuid4())[:8]
#     try:
#         table_html = table_data[j].get_attribute('outerHTML')
#     except (StaleElementReferenceException, WebDriverException):
#         table_data = driver.find_elements(By.XPATH, "//table")
#         table_html = table_data[j].get_attribute('outerHTML')
#     except IndexError:
#         break
#     if 'class="event"' in table_html:
#         page_events.append(tuple([id_])+get_event_data(table_html)+tuple([str(i+1)]))
#     if 'Дата-время скана линии' in table_html:
#         dict_page_odds[id_] = get_coefficients_data(table_html)
# df_page_events = pd.DataFrame(
#     columns=['id', 'date', 'players', 'event', 'score', 'winner', 'bc_page'],
#     data=page_events
# )
# df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
# dict_current_bc_odds.update(dict_page_odds)


# # 2
# i = 1
# npages = driver.find_elements(By.CLASS_NAME, 'pages')
# try:
#     driver.execute_script("arguments[0].click();", npages[i])
# except StaleElementReferenceException:
#     npages = driver.find_elements(By.CLASS_NAME, 'pages')
#     driver.execute_script("arguments[0].click();", npages[i])
# # iterating over all tables on page
# page_events = []
# dict_page_odds = {}
# table_data = driver.find_elements(By.XPATH, "//table")
# for j in tqdm(range(len(table_data))):
#     id_ = str(uuid.uuid4())[:8]
#     try:
#         table_html = table_data[j].get_attribute('outerHTML')
#     except (StaleElementReferenceException, WebDriverException):
#         table_data = driver.find_elements(By.XPATH, "//table")
#         table_html = table_data[j].get_attribute('outerHTML')
#     # except IndexError:
#     #     break
#     if 'class="event"' in table_html:
#         page_events.append(tuple([id_])+get_event_data(table_html)+tuple([str(i+1)]))
#     if 'Дата-время скана линии' in table_html:
#         dict_page_odds[id_] = get_coefficients_data(table_html)
# df_page_events = pd.DataFrame(
#     columns=['id', 'date', 'players', 'event', 'score', 'winner', 'bc_page'],
#     data=page_events
# )
# df_current_bc_events = pd.concat((df_current_bc_events, df_page_events))
# dict_current_bc_odds.update(dict_page_odds)

In [242]:
# dict_page_odds.keys()

In [243]:
# set(df_current_bc_events[df_current_bc_events['bc_page']=='1']['id']) & set(df_current_bc_events[df_current_bc_events['bc_page']=='2']['id'])

<h1>Что-то не вышло :(</h1>

Попробуем больше векторизации

In [268]:
from time import sleep 
from bs4 import BeautifulSoup

npages = driver.find_elements(By.CLASS_NAME, 'pages')
# pages iterating
for i in range(len(npages)):
    try:
        driver.execute_script("arguments[0].click();", npages[i])
    except StaleElementReferenceException:
        npages = driver.find_elements(By.CLASS_NAME, 'pages')
        driver.execute_script("arguments[0].click();", npages[i])    
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    html_soup = BeautifulSoup(html_source_code, 'html.parser')
    # KEYS TO SURVIVAL
    # html_soup.find_all("tbody")
    # html_soup.find_all("table", {"class": "event"})